In [1]:
# Import Required Packages
import torch
import os
import json
import sys
import re
import random
import importlib.util
from typing import *
from tqdm import tqdm 
from typing import List
import gc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap
import torch
import torch as nn

from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification,Trainer, TrainingArguments, PreTrainedModel,AutoConfig,DataCollatorWithPadding, PretrainedConfig
from peft import get_peft_model, LoraConfig
from datasets import Dataset

import gc


from JS_Architects import *

/home/jdelinea/anaconda3/envs/BarcHandbook/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
label_list = [
    "containment",
    "depth",
    "symmetry",
    "categorical",
    "spatial-Orientation",
    "spatial-Ordinal",
    "similarity",
    "quantitative",
    "replication",
    "figure-Ground",
    "continuity",
    "size",
    "closure",
    "centroid",
    "topological",
    "motion",
]

In [3]:
with open("perceptions_training.json", "r") as f:
    dic_training = json.load(f)

with open("perceptions_testing.json", "r") as f:
    dic_testing = json.load(f)

In [4]:
# %%
def labels_to_binary(label_list, input_labels):
    """
    Convert perceptions into a binary vector based on the label list.
    Handles both single strings and lists of strings for input_labels.
    """
    if isinstance(input_labels, str):
        input_labels = [input_labels]
    input_set = set(label.lower() for label in input_labels)
    return [1 if label.lower() in input_set else 0 for label in label_list]


def format_array(array, arr_sep="\n"):
    """
    Helper function to format a 2D array into a string with row-wise separation.
    """
    return arr_sep.join([" ".join(map(str, row)) for row in array])


def prepare_data_for_multilabel_classification(
    data_dict,
    instruction,
    label_list,
    inp_prefix="<I>",
    out_prefix="<O>",
    arr_sep="\n",
    exa_sep="\n---\n",
    bos_token="<|begin_of_text|>",
    eos_token="<|end_of_text|>"
):
    llama_data = []

    for entry_id, content in data_dict.items():
        # Convert text-based labels to a binary vector
        perceptions = content.get("perceptions", [])

        if isinstance(perceptions, str):
            perceptions = [perceptions]
   
        # Combine train and test examples (if you have them) 
        # or just use "train", depending on your data structure:
        examples = content.get("example", {}).get("train", []) + content.get("example", {}).get("test", [])

        # Format examples: we ONLY keep the input portion in the combined text
        # so that the model cannot trivially see the ground-truth output.
        formatted_examples = []
        for example in examples:
            # Input portion
            input_data = f"{inp_prefix}{format_array(example['input'], arr_sep)}"

            # Do NOT append the ground truth "output" arrays (for masking).
            # If you prefer to show partial or no output, adjust accordingly.
            formatted_examples.append(f"{input_data}")

        # Combine all input examples for this entry, with a BOS if desired
        combined_text = bos_token + exa_sep.join(formatted_examples) + eos_token

        llama_data.append({
            "instruction": instruction,
            "input": combined_text,   # masked so it doesn't include ground-truth outputs
            "output": perceptions,   # multi-label 0/1 vector
        })

    return llama_data


In [5]:
# %%
instruction1 = (
    "Classify the sequence of input-output pairs based on the following categories: "
    + ", ".join(label_list)
    + "."
)
# %%
# Prepare training data
llama_data_train = prepare_data_for_multilabel_classification(
    dic_training, instruction1, label_list
)
# Convert list-of-dict to Hugging Face Dataset
llama_data_dict_train = {
    "instruction": [item["instruction"] for item in llama_data_train],
    "input":       [item["input"]       for item in llama_data_train],
    "output":      [item["output"]      for item in llama_data_train],
}
train_dataset = Dataset.from_dict(llama_data_dict_train)

# Prepare testing data (if you want an eval set)
llama_data_test = prepare_data_for_multilabel_classification(
    dic_testing, instruction1, label_list
)
llama_data_dict_test = {
    "instruction": [item["instruction"] for item in llama_data_test],
    "input":       [item["input"]       for item in llama_data_test],
    "output":      [item["output"]      for item in llama_data_test],
}
test_dataset = Dataset.from_dict(llama_data_dict_test)

# Peek at first test example
train_dataset[0]

{'instruction': 'Classify the sequence of input-output pairs based on the following categories: containment, depth, symmetry, categorical, spatial-Orientation, spatial-Ordinal, similarity, quantitative, replication, figure-Ground, continuity, size, closure, centroid, topological, motion.',
 'input': '<|begin_of_text|><I>0 7 7\n7 7 7\n0 7 7\n---\n<I>4 0 4\n0 0 0\n0 4 0\n---\n<I>0 0 0\n0 0 2\n2 0 2\n---\n<I>6 6 0\n6 0 0\n0 6 6\n---\n<I>2 2 2\n0 0 0\n0 2 2\n---\n<I>7 0 7\n7 0 7\n7 7 0<|end_of_text|>',
 'output': ['spatial-ordinal', 'replication']}

In [6]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)


True
12.1


In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct.
403 Client Error. (Request ID: Root=1-6776a4b9-5a593a3b362286cd362c1a0d;8342bbd2-9903-4227-910b-865253727310)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct to ask for access.

In [24]:
special_tokens_dict = {
    "input": "<I>",
    "output": "<O>",
    "array_sep": "\n",
    "example_sep": "\n---\n",
    "eos_token": "<|end_of_text|>",
    "bos_token": "<|begin_of_text|>",
    "pad_token": "[PAD]"
}

# Add special tokens
tokenizer.add_special_tokens({
    "additional_special_tokens": [
        special_tokens_dict["input"],
        special_tokens_dict["output"],
        special_tokens_dict["array_sep"],
        special_tokens_dict["example_sep"]
    ],
    "eos_token": special_tokens_dict["eos_token"],
    "bos_token": special_tokens_dict["bos_token"],
    "pad_token": special_tokens_dict["pad_token"]
})

# Set the tokenizer pad token explicitly
tokenizer.pad_token = special_tokens_dict["pad_token"]

# Check the updated tokens
print(f"Special Tokens: {tokenizer.special_tokens_map}")
print(f"Vocabulary Size: {len(tokenizer)}")

# Resize model embeddings
model.resize_token_embeddings(len(tokenizer))


Special Tokens: {'bos_token': '<|begin_of_text|>', 'eos_token': '<|end_of_text|>', 'pad_token': '[PAD]', 'additional_special_tokens': ['<I>', '<O>', '\n', '\n---\n']}
Vocabulary Size: 128261


NameError: name 'model' is not defined

In [1]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [7]:
# Example LoRA config
lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
        "embed_tokens", "lm_head"
    ],
    lora_dropout=0.05,
    #bias="none",
    #task_type="CAUSAL_LM"  # or "SEQ_CLS" depending on usage
)

# Wrap your model with LoRA
model_classification = get_peft_model(model_classification, lora_config)


In [8]:
# %%
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["input"],
        padding="max_length",
        truncation=True,
        max_length=512,  # or 1024, depending on GPU memory
    )
    # Convert list-of-ints (0/1) to float for BCEWithLogitsLoss
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "labels": examples["output"],
    }

# Map over datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset  = test_dataset.map(tokenize_function, batched=True)


Map: 100%|██████████| 50/50 [00:00<00:00, 1313.96 examples/s]


In [9]:
# %%
# Remove original text columns to avoid confusion
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["instruction","input","output"])
tokenized_test_dataset  = tokenized_test_dataset.remove_columns(["instruction","input","output"])

# Set to torch
tokenized_train_dataset.set_format("torch")
tokenized_test_dataset.set_format("torch")

print(tokenized_train_dataset[0])


{'input_ids': tensor([128000, 128000, 128256,     15,    220,     22,    220,     22, 128258,
            22,    220,     22,    220,     22, 128258,     15,    220,     22,
           220,     22, 128259, 128256,     19,    220,     15,    220,     19,
        128258,     15,    220,     15,    220,     15, 128258,     15,    220,
            19,    220,     15, 128259, 128256,     15,    220,     15,    220,
            15, 128258,     15,    220,     15,    220,     17, 128258,     17,
           220,     15,    220,     17, 128259, 128256,     21,    220,     21,
           220,     15, 128258,     21,    220,     15,    220,     15, 128258,
            15,    220,     21,    220,     21, 128259, 128256,     17,    220,
            17,    220,     17, 128258,     15,    220,     15,    220,     15,
        128258,     15,    220,     17,    220,     17, 128259, 128256,     22,
           220,     15,    220,     22, 128258,     22,    220,     15,    220,
            22, 128258,   

-----

In [10]:
class CustomTrainer(Trainer):
    """
    A custom Trainer that overrides compute_loss to use BCEWithLogitsLoss 
    for multi-label classification.
    """
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")  # [batch_size, num_labels]
        outputs = model(**inputs)
        logits = outputs.logits  # [batch_size, num_labels]

        # Use BCEWithLogitsLoss for multi-label
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits.float(), labels.float())

        return (loss, outputs) if return_outputs else loss



In [11]:
# %%
# (Optional) Clear GPU cache before training
gc.collect()
torch.cuda.empty_cache()


training_args = TrainingArguments(
    output_dir="./JS_finetuned_model",
    evaluation_strategy="epoch",     # or "steps" / "no"
    learning_rate=1e-4,
    per_device_train_batch_size=1,   # adjust as you can
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    fp16=True,
    gradient_accumulation_steps=16,
    gradient_checkpointing=True,
    save_strategy="epoch",
    save_total_limit=2,
    #logging_dir="./logs",
    logging_steps=50,
    #report_to="tensorboard",         # if you use TensorBoard
)

data_collator = DataCollatorWithPadding(tokenizer, padding='longest')

trainer = CustomTrainer(
    model=model_classification,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,   # or None
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train
trainer.train()

# Save final model
trainer.save_model("./fine_tuned_llama_multilabel")
tokenizer.save_pretrained("./fine_tuned_llama_multilabel")

# Clear memory if needed
gc.collect()
torch.cuda.empty_cache()



/home/jdelinea/anaconda3/envs/BarcHandbook/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


/tmp/ipykernel_629082/3624235862.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


[2025-01-01 10:45:13,331] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/jdelinea/anaconda3/envs/BarcHandbook/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/jdelinea/anaconda3/envs/BarcHandbook/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda-12.6/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/jdelinea/anaconda3/envs/BarcHandbook/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda-12.6/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/jdelinea/anaconda3/envs/BarcHandbook/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda-12.6/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/jdelinea/anaconda3/envs/BarcHandbook/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda-12.6/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/jdelinea/anaconda3/envs/BarcHandbook/compiler_compat/ld: /usr/local/cuda-12.6/lib64/libcufile.so: undefined reference t

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [ ]:
# %%
predictions = trainer.predict(tokenized_test_dataset)
logits = predictions.predictions
# Sigmoid to get probabilities
probs = torch.sigmoid(torch.tensor(logits))
# Convert probabilities to 0/1 predictions with a threshold
threshold = 0.5
pred_labels = (probs > threshold).int()
print("Predicted binary labels shape:", pred_labels.shape)


In [ ]:
break

In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")

# Add special tokens if required
special_tokens_dict = {
    "input": "<I>",
    "output": "<O>",
    "array_sep": "\n",
    "example_sep": "\n---\n",
    "eos_token": "<|end_of_text|>",
    "bos_token": "<|begin_of_text|>",
    "pad_token": "[PAD]"
}

tokenizer.add_special_tokens({
    "additional_special_tokens": [
        special_tokens_dict["input"],
        special_tokens_dict["output"],
        special_tokens_dict["array_sep"],
        special_tokens_dict["example_sep"]
    ],
    "eos_token": special_tokens_dict["eos_token"],
    "bos_token": special_tokens_dict["bos_token"],
    "pad_token": special_tokens_dict["pad_token"]
})

# Set the tokenizer pad token explicitly
tokenizer.pad_token = special_tokens_dict["pad_token"]

# Check the updated tokens
print(f"Special Tokens: {tokenizer.special_tokens_map}")
print(f"Vocabulary Size: {len(tokenizer)}")

In [ ]:
test_sequence = "<I>8 6\n6 4\n---\n<O>8 6 8 6 8 6\n6 4 6 4 6 4\n6 8 6 8 6 8\n4 6 4 6 4 6\n8 6 8 6 8 6\n6 4 6 4 6 4<|end_of_text|>\n---\n<I>7 9\n4 3\n---\n<O>7 9 7 9 7 9\n4 3 4 3 4 3\n9 7 9 7 9 7\n3 4 3 4 3 4\n7 9 7 9 7 9\n4 3 4 3 4 3<|end_of_text|>\n---\n<I>3 2\n7 8\n---\n<O>3 2 3 2 3 2\n7 8 7 8 7 8\n2 3 2 3 2 3\n8 7 8 7 8 7\n3 2 3 2 3 2\n7 8 7 8 7 8<|end_of_text|>"
encoded = tokenizer.encode(test_sequence)
decoded = tokenizer.decode(encoded)
print("Encoded:", encoded)
print("Decoded:", decoded)

In [ ]:
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["input"],
        padding="max_length",
        truncation=True,
        max_length=2048  # Reduced from 8192 to 2048
    )
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "labels": examples["output"]
    }


# Keep only the input output no instructions for now
tokenized_train_dataset = llama_data_dataset.map(tokenize_function, batched=True)

tokenized_train_dataset

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn


# DataLoader
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Training
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = nn.CrossEntropyLoss()

model.train()
for epoch in range(3):  # Adjust number of epochs as needed
    for batch in dataloader:
        input_ids = torch.stack(batch["input_ids"]).to(gpu)
        labels = torch.stack(batch["labels"]).to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, labels=labels)
        logits = outputs.logits[:, -1, :]  # Get logits for the label token
        loss = loss_fn(logits, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}: Loss = {loss.item()}")

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_llama")
tokenizer.save_pretrained("./fine_tuned_llama")

In [ ]:
# %% [markdown]
# ## 6. Train - Optimized

# %%
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
import torch
import gc

# Clear cache before training
gc.collect()
torch.cuda.empty_cache()

# Define TrainingArguments with optimizations
training_args = TrainingArguments(
    output_dir="./JS_finetuned_model",
    evaluation_strategy="no",
    learning_rate=1e-4,
    per_device_train_batch_size=1,  # Reduced batch size
    num_train_epochs=3,  # Increased epochs if feasible
    fp16=True,
    gradient_accumulation_steps=8,  # Increased gradient accumulation
    gradient_checkpointing=True,  # Enable gradient checkpointing
    save_strategy="epoch",
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50,
    report_to="tensorboard",
    # Add any other necessary arguments
)

# Use built-in data collator with dynamic padding
data_collator = DataCollatorWithPadding(tokenizer, padding='longest')

# Initialize Trainer
trainer = CustomTrainer(
    model=model_classification,
    args=training_args,
    train_dataset=tokenized_final_dataset,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Start Training
trainer.train()

# Clear memory after training
gc.collect()
torch.cuda.empty_cache()
